In [1]:
import sys
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [2]:
year = 2018
# 각 년도 및 기록 웹페이지를 딕셔너리 형태로 저장
# 타격확장 및 투수확장의 경우 규정타석/규정이닝이 아닌 전체타석/전체이닝으로 설정
record_site = {'타격기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'타격확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=WRCPLUS&o2=WAR_ALL&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP&o2=WAR&de=0&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수구속':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=14&o1=FVval&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
}

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\admin\.wdm\drivers\chromedriver\win32\105.0.5195.52]


In [4]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['타격기본'])
hit_basic_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 hit_basic_df_list에 추가
    hit_basic_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [5]:
hit_basic_df = pd.concat(hit_basic_df_list)
hit_basic_df

순    이름      팀     정렬    G   타석   타수   득점   안타  2타  ... 희타 희비     비율  \
      순    이름      팀   WAR*    G   타석   타수   득점   안타  2타  ... 희타 희비     타율   
0     1   김재환  18두LF   6.94  139  602  527  104  176  36  ...  0  7   .334   
1     2   박병호  18넥1B   6.62  113  488  400   88  138  20  ...  0  3   .345   
2     3   양의지   18두C   6.42  133  503  439   84  157  29  ...  0  6   .358   
3     4   전준우  18롯LF   5.67  144  614  556  118  190  36  ...  1  2   .342   
4     5   로하스  18KCF   5.66  144  645  564  114  172  30  ...  1  2   .305   
..  ...   ...    ...    ...  ...  ...  ...  ...  ...  ..  ... .. ..    ...   
30  327  파레디스  18두RF  -0.73   21   71   65    7    9   2  ...  0  1   .138   
31  328   이상호  18N2B  -0.73  114  204  187   32   47   2  ...  7  0   .251   
32  329   박기혁  18KSS  -0.80   98  196  173   17   37   6  ...  5  3   .214   
33  330   김용의  18L1B  -0.85  114  185  159   28   37   2  ...  7  3   .233   
0   331   나종덕   18롯C  -1.71  106  203  177   15   22   3  ...  9  0  0.124   

                                        WAR*    WPA  
       출루     장타    OPS   wOBA   wRC+   WAR*    WPA  
0    .405   .657  1.062   .438  168.8   6.94   5.80  
1    .457   .718  1.175   .478  190.9   6.62   6.44  
2    .427   .585  1.013   .431  160.5   6.42   3.91  
3    .400   .592   .991   .421  146.3   5.67   5.45  
4    .388   .590   .979   .410  138.5   5.66   5.41  
..    ...    ...    ...    ...    ...    ...    ...  
30   .197   .246   .443   .191   -2.4  -0.73  -0.65  
31   .289   .262   .551   .251   37.3  -0.73  -1.23  
32   .272   .266   .538   .244   28.9  -0.80  -1.86  
33   .298   .245   .543   .248   33.0  -0.85  -0.84  
0   0.201  0.175  0.376  0.175  -15.1  -1.71  -1.85  

[375 rows x 31 columns]

In [6]:
driver.get(record_site['타격확장'])
hit_expand_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 hit_expand_df_list에 추가
    hit_expand_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [7]:
hit_expand_df = pd.concat(hit_expand_df_list)
hit_expand_df

순   이름      팀   생산력+   타석    HR%   BB%    K%  BB/K  IsoP  ...   PSN  \
      순   이름      팀   wRC+   타석    HR%   BB%    K%  BB/K  IsoP  ...   PSN   
0     1  백상원  18삼2B  219.5    8   0.00  25.0   0.0   NaN  .000  ...   NaN   
1     2  유재신  18KLF  208.2   40   2.50  17.5  25.0  0.70  .242  ...  1.60   
2     3  박병호  18넥1B  190.9  488   8.81  13.9  23.4  0.60  .373  ...  0.00   
3     4  김종덕   18넥C  174.6    2   0.00   0.0   0.0   NaN  .000  ...   NaN   
4     5   샌즈  18넥RF  171.5   93  12.90   6.5  29.0  0.22  .454  ...  0.00   
..  ...  ...    ...    ...  ...    ...   ...   ...   ...   ...  ...   ...   
30  NaN  구승민   18롯P    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
31  NaN  이동걸   18한P    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
32  NaN  홍건희   18KP    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
33  NaN  브리검   18넥P    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
0   NaN  송창식   18한P    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   

   타격 생산력                     타격 생산력 (파크팩터 조정)                             
     wOBA    wRC wRC/27  wRAA             wOBA    wRC wRC/27  wRAA   wRC+  
0    .526    2.4  22.02   1.3             .529    2.5  22.20   1.3  219.5  
1    .512   11.7  16.62   6.1             .512   11.7  16.62   6.1  208.2  
2    .478  127.4  12.50  58.8             .486  130.9  12.85  62.3  190.9  
3    .460    0.5  13.17   0.2             .461    0.5  13.24   0.2  174.6  
4    .445   21.4   9.17   8.3             .457   22.4   9.60   9.3  171.5  
..    ...    ...    ...   ...              ...    ...    ...   ...    ...  
30    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
31    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
32    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
33    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
0     NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  

[375 rows x 23 columns]

In [8]:
# columns의 level 0을 없애고 정렬을 위해 4번째 위치에 자리잡은 열 제거하는 함수 생성
def table_change(df):
    df = df.droplevel(0, axis=1)
    columns = df.columns.tolist()
    # pop() 메서드로 제거하면 기본에서는 WAR*, 확장에서는 wRC+가 중복으로 발생하는데.. 왜지?
    # del 제거도 마찬가지
    del columns[3]
    df = df.loc[:, columns]
    return df

In [9]:
# DataFrame에 있는 불필요한 행 제거하는 함수 생성
def remove_records(df):
    df = df.query("이름 != '이름'")
    return df

In [10]:
# 기본적으로 크롤링 데이터는 str 형식이기에 숫자로 바꿔줄 필요 -> pd.to_numeric 활용할 함수 작성
# 열 정리를 다 하고 새로운 객체로 할당한 이후 적용 가능
def numeric(df):
    for col in df.columns.tolist():
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

In [16]:
hit_basic_cols = ['순', '이름', '팀', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런',
       '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비',
       '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA']
hit_basic_df2 = (hit_basic_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, hit_basic_cols]
)
# 중복되어 나오는 문제를 해결하기 위해 열 drop 후 새로운 Series로 추가
hit_basic_war = hit_basic_df2.loc[:, 'WAR*'].iloc[:, 0]
hit_basic_df3 = (hit_basic_df2
.drop(['WAR*', '순'], axis=1)
.assign(**{'WAR*':hit_basic_war})
)
hit_basic_df3

,이름,팀,G,타석,타수,득점,안타,2타,3타,홈런,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WPA,WAR*
0,김재환,18두LF,139,602,527,104,176,36,1,44,...,0,7,.334,.405,.657,1.062,.438,168.8,5.80,6.94
1,박병호,18넥1B,113,488,400,88,138,20,0,43,...,0,3,.345,.457,.718,1.175,.478,190.9,6.44,6.62
2,양의지,18두C,133,503,439,84,157,29,1,23,...,0,6,.358,.427,.585,1.013,.431,160.5,3.91,6.42
3,전준우,18롯LF,144,614,556,118,190,36,2,33,...,1,2,.342,.400,.592,.991,.421,146.3,5.45,5.67
4,로하스,18KCF,144,645,564,114,172,30,1,43,...,1,2,.305,.388,.590,.979,.410,138.5,5.41,5.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,파레디스,18두RF,21,71,65,7,9,2,1,1,...,0,1,.138,.197,.246,.443,.191,-2.4,-0.65,-0.73
31,이상호,18N2B,114,204,187,32,47,2,0,0,...,7,0,.251,.289,.262,.551,.251,37.3,-1.23,-0.73
32,박기혁,18KSS,98,196,173,17,37,6,0,1,...,5,3,.214,.272,.266,.538,.244,28.9,-1.86,-0.80
33,김용의,18L1B,114,185,159,28,37,2,0,0,...,7,3,.233,.298,.245,.543,.248,33.0,-0.84,-0.85


In [17]:
hit_expand_cols = ['순', '이름', '팀', '타석', 'HR%', 'BB%', 'K%', 'BB/K', 'IsoP', 'IsoD',
       'BABIP', 'Spd', 'PSN', 'wRC+']
hit_expand_df2 = (hit_expand_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, hit_expand_cols]
)
# 중복되어 나오는 문제를 해결하기 위해 열 drop 후 새로운 Series로 추가
hit_expand_wrc = hit_expand_df2.loc[:, 'wRC+'].iloc[:, 0]
hit_expand_df3 = (hit_expand_df2
.drop(['wRC+', '순'], axis=1)
.assign(**{'wRC+':hit_expand_wrc})
)
hit_expand_df3

,이름,팀,타석,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN,wRC+
0,백상원,18삼2B,8,0.00,25.0,0.0,NaN,.000,.125,.500,2.4,NaN,219.5
1,유재신,18KLF,40,2.50,17.5,25.0,0.70,.242,.101,.591,8.0,1.60,208.2
2,박병호,18넥1B,488,8.81,13.9,23.4,0.60,.373,.112,.386,1.7,0.00,190.9
3,김종덕,18넥C,2,0.00,0.0,0.0,NaN,.000,.000,.500,1.9,NaN,174.6
4,샌즈,18넥RF,93,12.90,6.5,29.0,0.22,.454,.041,.313,3.8,0.00,171.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,구승민,18롯P,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,이동걸,18한P,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,홍건희,18KP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,브리검,18넥P,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# 타격-기본과 타격-확장을 합쳐주기 위해 merge 활용
hit_df = (hit_basic_df3
.merge(hit_expand_df3, on=['이름', '팀', '타석', 'wRC+'], how='inner')
.pipe(numeric)
)
hit_df.columns

Index(['이름', '팀', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
       '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타',
       'OPS', 'wOBA', 'wRC+', 'WPA', 'WAR*', 'HR%', 'BB%', 'K%', 'BB/K',
       'IsoP', 'IsoD', 'BABIP', 'Spd', 'PSN'],
      dtype='object')

In [22]:
(hit_df
.loc[:, '팀']
.str.slice(0, 2)
)

0      18
1      18
2      18
3      18
4      18
       ..
326    18
327    18
328    18
329    18
330    18
Name: 팀, Length: 331, dtype: object

In [24]:
hit_columns = ['이름', '연도', '소속', '포지션', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
       '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타',
       'OPS', 'wOBA', 'wRC+', 'WPA', 'WAR*', 'HR%', 'BB%', 'K%', 'BB/K',
       'IsoP', 'IsoD', 'BABIP', 'Spd', 'PSN']

In [30]:
(hit_df
.assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
소속=lambda df: df['팀'].str.extract('(L|키|K|S|N|한|삼|롯|두|넥)')
.replace({'L':'LG', '키':'넥센/키움', 'S':'SK/SSG', 'N':'NC', '한':'한화', '삼':'삼성', '롯':'롯데', '두':'두산', '넥':'넥센/키움'}),
포지션=lambda df: df['팀'].str.extract('(LF|CF|RF|1B|2B|3B|SS|C|DH|P)'))
.loc[:, hit_columns]
)

,이름,연도,소속,포지션,G,타석,타수,득점,안타,2타,...,WAR*,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,김재환,2018,두산,LF,139,602,527,104,176,36,...,6.94,7.31,9.8,22.3,0.44,0.323,0.071,0.371,3.5,3.83
1,박병호,2018,넥센/키움,1B,113,488,400,88,138,20,...,6.62,8.81,13.9,23.4,0.60,0.373,0.112,0.386,1.7,0.00
2,양의지,2018,두산,C,133,503,439,84,157,29,...,6.42,4.57,9.0,8.0,1.13,0.228,0.070,0.351,3.9,9.52
3,전준우,2018,롯데,LF,144,614,556,118,190,36,...,5.67,5.38,7.8,13.4,0.59,0.250,0.058,0.354,4.1,11.55
4,로하스,2018,K,CF,144,645,564,114,172,30,...,5.66,6.67,11.0,22.0,0.50,0.286,0.083,0.339,4.2,25.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,이종욱,2018,NC,LF,43,128,122,12,28,7,...,-0.71,0.78,3.1,12.5,0.25,0.082,0.022,0.255,3.4,1.00
327,파레디스,2018,두산,RF,21,71,65,7,9,2,...,-0.73,1.41,5.6,23.9,0.24,0.108,0.059,0.167,6.8,1.00
328,이상호,2018,NC,2B,114,204,187,32,47,2,...,-0.73,0.00,3.9,7.4,0.53,0.011,0.038,0.273,5.0,0.00
329,박기혁,2018,K,SS,98,196,173,17,37,6,...,-0.80,0.51,6.6,25.5,0.26,0.052,0.058,0.288,1.9,1.00


In [32]:
kt_site = {'타격':f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys={year}&ye={year}&se=0&te=kt&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=0&si=&cn=',
'투수':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=kt&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='}

In [33]:
kt_df_list = []
driver.get(kt_site['타격'])
while True:
    page = driver.page_source
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 kt_df_list에 추가
    kt_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [39]:
kt_team_member = (pd.concat(kt_df_list)
.pipe(table_change)
['이름']
.tolist()
)
kt_team_member

['로하스',
 '유한준',
 '황재균',
 '박경수',
 '강백호',
 '장성우',
 '윤석민',
 '이진영',
 '이대형',
 '고창성',
 '이름',
 '이름',
 '이종혁',
 '김사율',
 '엄상백',
 '신병률',
 '김재윤',
 '홍성용',
 '홍현빈',
 '이준수',
 '고명성',
 '정현',
 '이름',
 '이름',
 '심우준',
 '오태곤',
 '김영환',
 '배정대',
 '김사연',
 '남태혁',
 '이해창',
 '김동명',
 '오정복',
 '이창진',
 '전민수',
 '김진곤',
 '하준호',
 '오준혁',
 '송민섭',
 '박기혁']

In [66]:
(hit_df
.assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
소속=lambda df: df['팀'].str.extract('(L|키|K|S|N|한|삼|롯|두|넥)')
.replace({'L':'LG', '키':'넥센/키움', 'K':'KIA', 'S':'SK/SSG', 'N':'NC', '한':'한화', '삼':'삼성', '롯':'롯데', '두':'두산', '넥':'넥센/키움'}),
포지션=lambda df: df['팀'].str.extract('(LF|CF|RF|1B|2B|3B|SS|C|DH|P)'))
# in 대신 .isin()메서드를 써야하는 걸 한참을 삽질..
.assign(소속=lambda df: np.where(df['이름'].isin(kt_team_member), 'KT', df['소속']))
.loc[:, hit_columns]
)

,이름,연도,소속,포지션,G,타석,타수,득점,안타,2타,...,WAR*,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,김재환,2018,두산,LF,139,602,527,104,176,36,...,6.94,7.31,9.8,22.3,0.44,0.323,0.071,0.371,3.5,3.83
1,박병호,2018,넥센/키움,1B,113,488,400,88,138,20,...,6.62,8.81,13.9,23.4,0.60,0.373,0.112,0.386,1.7,0.00
2,양의지,2018,두산,C,133,503,439,84,157,29,...,6.42,4.57,9.0,8.0,1.13,0.228,0.070,0.351,3.9,9.52
3,전준우,2018,롯데,LF,144,614,556,118,190,36,...,5.67,5.38,7.8,13.4,0.59,0.250,0.058,0.354,4.1,11.55
4,로하스,2018,KT,CF,144,645,564,114,172,30,...,5.66,6.67,11.0,22.0,0.50,0.286,0.083,0.339,4.2,25.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,이종욱,2018,NC,LF,43,128,122,12,28,7,...,-0.71,0.78,3.1,12.5,0.25,0.082,0.022,0.255,3.4,1.00
327,파레디스,2018,두산,RF,21,71,65,7,9,2,...,-0.73,1.41,5.6,23.9,0.24,0.108,0.059,0.167,6.8,1.00
328,이상호,2018,NC,2B,114,204,187,32,47,2,...,-0.73,0.00,3.9,7.4,0.53,0.011,0.038,0.273,5.0,0.00
329,박기혁,2018,KT,SS,98,196,173,17,37,6,...,-0.80,0.51,6.6,25.5,0.26,0.052,0.058,0.288,1.9,1.00


In [ ]:
# 투수 쪽 진행..